<a href="https://colab.research.google.com/github/tiendatngcs/NLP/blob/master/Vietnamese_news_topic_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from bs4 import BeautifulSoup
import requests
import pickle as pkl
import tqdm

## Gathering data

We scrape the web with BeautifulSoup



Prototype:

In [38]:
url = 'https://vnexpress.net/hang-khong-hut-nguon-thu-cao-diem-he-4152498.html'

In [39]:
markup = requests.get(url).text
soup = BeautifulSoup(markup=markup, features="lxml")

In [40]:
text = [p.text for p in soup.find(name="section", class_="section page-detail top-detail").find(name="article", class_="fck_detail ").find_all("p", class_="Normal")]

In [41]:
text

['Sau khi dịch bệnh được kiểm soát hồi cuối tháng 4, hàng không Việt dần tăng khai thác trở lại các chặng nội địa từ tháng 5. Tiếp đà phục hồi nhanh chóng, các hãng rất kỳ vọng sẽ bù đắp được dòng tiền nhờ cao điểm hè từ tháng 7 đến tháng 9. Tuy nhiên, làn sóng dịch thứ hai bùng phát từ cuối tháng 7 đã chuyển giai đoạn cao điểm thành thấp điểm.',
 'Theo thống kê của Cục Hàng không giai đoạn 19/7 – 18/8, tổng số chuyến bay của 5 hãng trong nước chỉ còn hơn 16.400, giảm 33% so với tháng trước và 45% so với cùng kỳ năm ngoái. Trong đó, Vietnam Airlines bị giảm hơn 4.300 chuyến, tương đương giảm gần 40% so với tháng 7, còn khoảng 6.700 chuyến. Vietjet Air khai thác hơn 5.700 chuyến, giảm 31%, tương đương trên 2.600 chuyến bay.',
 'Tổng số chuyến của các hãng Bamboo Airways, Pacific Airways, VASCO giảm lần lượt giảm 17%, 15% và 43% so với giai đoạn tháng 7. Các hãng bay đã hủy 490 chuyến, tăng 325 chuyến so với tháng trước đó. Các hãng gần đây của cũng phải giảm tần suất hoặc dừng khai thác

Gather article urls

In [132]:
topic_urls = [
              "https://vnexpress.net/kinh-doanh", # business
              "https://vnexpress.net/giai-tri", # entertainment
              "https://vnexpress.net/the-thao", # sports
              "https://vnexpress.net/phap-luat", # laws
              "https://vnexpress.net/giao-duc", # education
              "https://vnexpress.net/suc-khoe", # health
              "https://vnexpress.net/doi-song", # life
              "https://vnexpress.net/du-lich", # travel
              "https://vnexpress.net/khoa-hoc", # science
              "https://vnexpress.net/so-hoa", # digital
              "https://vnexpress.net/oto-xe-may", # vehicles
              "https://vnexpress.net/hai"] # comedy
pages = [""] + ["/p"+ str(i) for i in range(2, 20)]

In [128]:
def get_articles_links(topic_link:str):
  markup = requests.get(topic_link).text
  soup = BeautifulSoup(markup=markup, features="lxml")
  
  # article_tags = soup.find_all("article", class_="item-news item-news-common ")
  # if len(article_tags) == 0:
  #   article_tags = soup.find_all("article", class_="item-news item-news-common")
  # if len(article_tags) == 0:
  #   return []
  # urls = [article_tag.find("a", href=True)["href"] for article_tag in article_tags]
  urls = []
  for tag in soup.find_all("article"):
    a_tag = tag.find("a", href=True)
    if a_tag is not None:
      urls.append(a_tag["href"])
  # urls = vne_soup.find("article", class_="item-news item news-common").find_all(a, href=True)
  return urls

In [133]:
for topic_url in topic_urls:
  print(topic_url)
  get_articles_links(topic_url)

https://vnexpress.net/kinh-doanh
https://vnexpress.net/giai-tri
https://vnexpress.net/the-thao
https://vnexpress.net/phap-luat
https://vnexpress.net/giao-duc
https://vnexpress.net/suc-khoe
https://vnexpress.net/doi-song
https://vnexpress.net/du-lich
https://vnexpress.net/khoa-hoc
https://vnexpress.net/so-hoa
https://vnexpress.net/oto-xe-may
https://vnexpress.net/hai


In [134]:
articles = {}
for i, topic_url in enumerate(topic_urls[1:]):
  articles[i] = []
  for page in pages:
    url = topic_url + page
    print(url)
    articles[i] = articles[i] + get_articles_links(url)


https://vnexpress.net/giai-tri
https://vnexpress.net/giai-tri/p2
https://vnexpress.net/giai-tri/p3
https://vnexpress.net/giai-tri/p4
https://vnexpress.net/giai-tri/p5
https://vnexpress.net/giai-tri/p6
https://vnexpress.net/giai-tri/p7
https://vnexpress.net/giai-tri/p8
https://vnexpress.net/giai-tri/p9
https://vnexpress.net/giai-tri/p10
https://vnexpress.net/giai-tri/p11
https://vnexpress.net/giai-tri/p12
https://vnexpress.net/giai-tri/p13
https://vnexpress.net/giai-tri/p14
https://vnexpress.net/giai-tri/p15
https://vnexpress.net/giai-tri/p16
https://vnexpress.net/giai-tri/p17
https://vnexpress.net/giai-tri/p18
https://vnexpress.net/giai-tri/p19
https://vnexpress.net/the-thao
https://vnexpress.net/the-thao/p2
https://vnexpress.net/the-thao/p3
https://vnexpress.net/the-thao/p4
https://vnexpress.net/the-thao/p5
https://vnexpress.net/the-thao/p6
https://vnexpress.net/the-thao/p7
https://vnexpress.net/the-thao/p8
https://vnexpress.net/the-thao/p9
https://vnexpress.net/the-thao/p10
https://v

Save the article links

In [135]:
with open("/content/drive/My Drive/articles.pkl", "wb") as outfile:
  pkl.dump(articles, outfile, pkl.HIGHEST_PROTOCOL)

Load the article links

In [136]:
with open("/content/drive/My Drive/articles.pkl", "rb") as infile:
  load_articles = pkl.load(infile)

In [141]:
for key in articles.keys():
  print(len(articles[key]))

584
580
152
143
587
575
575
143
583
156
143


In [87]:
articles.keys()

dict_keys([0, 1])

In [ ]:
vne = "https://vnexpress.net/"
vne_page = requests.get(vne).text
vne_soup = BeautifulSoup(markup=vne_page, features="lxml")
vne_soup_menu = vne_soup.find_all("section", class_="wrap-all-menu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
vne_soup_menu

[<section class="wrap-all-menu"></section>]